In [1]:
import onnxruntime as rt
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes  # noqa
from onnxmltools.convert.lightgbm.operator_converters.LightGbm import convert_lightgbm  # noqa
from skl2onnx.common.data_types import FloatTensorType, DoubleTensorType
import numpy as np
from lightgbm import LGBMClassifier
from sqlalchemy import create_engine
import sqlalchemy as db
import pandas as pd
import random 
from sklearn.model_selection import train_test_split

from mclLib.featurenames import *
from mclLib.cleaner import *
from mclLib.randomer import *
from mclLib.server_info import *

In [2]:

engine = create_engine(mysql_server_uri)
conn = engine.connect()


In [3]:
''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [4]:
''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) & ( br_full_data['dTradeTime'] < '2023-03-02' )
br = br_full_data[get_filter]

In [5]:
''' get features name'''
feature_names =  f_name_102
feature_size = len(feature_names)

''' set X data '''
X = br[feature_names].to_numpy(dtype=np.float64)

''' set y data '''
y_condition = (br['fMaxPowerAfterBuyWhile30'] < 0.03)
y = np.where(y_condition, 1, 0)


In [6]:
''' make random seed '''
random_seed = getRandomSeed()

''' split train test validation data '''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

In [17]:
lgbmc = LGBMClassifier(feature_fraction=0.8, bagging_freq=10)

In [18]:
lgbmc.fit(X_train, y_train)

LGBMClassifier(bagging_freq=10, feature_fraction=0.8)

In [8]:
update_registered_converter(
    LGBMClassifier, 'LightGbmLGBMClassifier',
    calculate_linear_classifier_output_shapes, convert_lightgbm,
    options={'nocl': [True, False], 'zipmap': [True, False, 'columns']})

In [9]:
model_onnx = convert_sklearn(
    lgbmc, 'pipeline_lightgbm',options= {'zipmap': False},
    initial_types=[('input', DoubleTensorType([None, 102]))],
    final_types=[('output', DoubleTensorType([None])),
                ('prob', DoubleTensorType([None, 2]))],
    target_opset={'': 17, 'ai.onnx.ml': 2})

# And save.
with open("./onnx/lgbm30_d3.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

In [10]:
cleanAll()